# Site EM Lab Assignment

## Introduction

In this lab you will implement the expectation maximization part of the MEME+
algorithm described by Bailey and Elkan in their technical report titled
["Fitting a Mixture Model by Expectation Maximization to Discover Motifs in BioPolymers."](https://cdn.aaai.org/ISMB/1994/ISMB94-004.pdf)

The outer loops, initialization, and the various "hacks" they added to the basic
EM model are provided for you, so you can focus on the EM part. However, it is
still the most code you've had to write so far, by a significant amount.



## Files

As with the previous assignments, you will be asked to write code in
[siteEM/assignment.py](assignment.py). The sections where you need to
write code are identified, as they have been in the past, with
`raise NotImplementedError`.

There are also some additional files and directories in this assignment that
did not exist in previous assigments:

- test data is provided in [siteEM/data](data)

- The Bailey and Elkan hacks, and some other convenience functions which you
will not be asked to implemented, are in [siteEM/utils](utils).

- [siteEM/\_\_main\_\_.py](__main__.py) provides a module entry point
which allows you to run the code in the package as a script.

## Running the code in a notebook

By including the right import statements in a notebook cell, you can
call `siteEM_initializer` and/or `siteEM` in a notebook. You can use 
[siteEM/test_assignment.py](test_assignment.py) as a guide for how to do this.


## Data representations

### Input list of sequences

The sequences passed to the site EM algorithm are represented as lists of
nucleotides. The nucleotides are represented by the integers 0, 1, 2, 3 with
the following mapping:

```raw
A <-> 0
C <-> 1
G <-> 2
T <-> 3
```

You should use these as indices into the probability frequency matrix (PFM).

The total input is a list of sequences, where each sequence is a list of integers.

### Model parameters

The parameters related to the motif for which we search which are passed to the
site EM algorithm are stored in a
[SequenceModel](https://cse587a.github.io/cse587Autils/SequenceObjects/API/SequenceModel.html)
object. Briefly, this object stores the prior probability (`site_prior`) of 
observing a motif in a sequence, and the complement, `background_prior`, which
is the probability of observing a non-motif sequence that is generated from a 
background distribution on nucleotides. A `SequenceModel` object also stores 
Position Frequency Matricies (PFMs) for the motif and the background distribution.

- The motif PFM, stored in the attribute `site_base_probs` is represented as a
list of list where each sublist is length 4 with indicies corresponding to the
nucleotide mapping above. Each sublist represents a position (typically called a
column based on the way PFM are commonly visualized) in the motif and describes
the relative frequency of each nucleotide at that position.

- The background PFM, stored in the attribute `background_base_probs` is a
single list of length 4 where each index corresponds to the nucleotide mapping
above. This PFM describes the relative frequency of each nucleotide in the
background. This is assumed to be the same for all positions in the motif.

### Posteriors and erasers

- The purpose of the `e_step()` is to calculate the **posteriors**.
This is important: The posterior probability of a motif of length motif_length
starting at each position in a single input sequence is represented as a numpy 
array with length sequence_length - motif_length + 1. In the code, this is
sometimes call a `posteriors_row` The posteriors for the 
entire input is a list of these. The kth posterior in a `posteriors_row` is the 
posterior probability for the sequence from k to k + motif_length - 1, inclusive. 
In this implementation, each posteriors row is normalized so the probabilities
some to one, corresponding to the assumption of one motif occurrence per input
sequence. Some of the Bailey and Elkan papers specify an alternative normalization
where the user supplies an expected number of motif occurrences in the entire
input and the posterior probabilities are normalized so they sum to that number
across the entire input.

- Erasers is a list of numpy arrays with the same dimensions as input. The 
entries are initialized to 1 and  may have any value between 0 and 1, inclusive. 
They are multiplicative factors that reduce the
expected counts of letters. Let  m be the number of the  PFM column under
consideration. If letter k occurs in position i+m-1 of sequence j, it would
normally contribute posteriors[i, j] to the expected to the expected count for
letter k in column m of the PFM. With erasers, it will contribute posteriors[i,
j] * erasers[i, j+m-1]. Bailey and Elkan are not very clear about this and 
different papers suggest different ways of doing it. Eraser updates are calculated
for you but you will need to use the eraser values to modify the expected counts
as above.

### Return values
The top level function, `siteEM_initializer`, returns list of pairs, each pair
being the result of searching for one motif 
```
[(motif1_posteriors, motif1_sequence_model),
...
 (motifn_posteriors, motifn_sequence_model)]
```
The posteriors are provided for debugging convenience -- usually you will just
look at the sequence model. Even when only one motif is requested, it will be 
returned in a list containing only one pair.

The most likely single sequence can be printed out by calling `consensus_pfm` from
the utils directory.

In general, it is worth looking at the tests carefully to see which modules need
to be imported, how the functions in the codebase are called, and what they're 
expected to return before you start coding.

## Questions
Please answer the following questions in a new cell below the question. Don't start this part until your code passes all the tests provided. You may have to instrument your code a little to get these answers out. If so, retest to make sure your code still passes all the tests.

1. **Convergence.** Using the sequence file smallTest.fasta, how many iterations does it take make MM to converge with accuracy 0.01 and no effective limit on the number of iterations?

In [ ]:
from importlib.resources import path
from importlib import reload
import siteEM.assignment as assign
reload(assign) # This ensures code is reloaded each time
from siteEM.assignment import siteEM_intializer
with path('siteEM.data', 'smallTest.fasta') as small_fasta_path:
            fasta_path = small_fasta_path
# Instrument your code to print out the iteration number and call 
# `siteEM_initializer`` to answer the question.

2. **Dependence on initialization.** Using the sequence file smallTest.fasta with motif width 4, max iterations 100, and accuracy 0.01, run siteEM_initializer 10 times in a loop with seed=0 to 9. How many different consensus motifs did you get? If you got more than one, how similar were they?

In [ ]:
from importlib.resources import path
from importlib import reload
import siteEM.assignment as assign
reload(assign) # This ensures code is reloaded each time
from siteEM.assignment import siteEM_intializer
from siteEM.utils import consensus_pfm
with path('siteEM.data', 'smallTest.fasta') as small_fasta_path:
            fasta_path = small_fasta_path

# Loop through seeds, call siteEM_initializer, and print out consensus motifs.
# Then write some text to answer the question.

3. **Comparison to the online MEME on real promoters.** There is a very fancy and heavily engineered MEME implementation available online at [http://meme-suite.org/tools/meme](http://meme-suite.org/tools/meme). The file PACPlusSeqs.fasta contains segments of the promoters of yeast genes most which encode ribosomal proteins. They are admittedly carefully chose, and these segments (between 100 and 300 bp upstream of the start codon) are known to contain most of the instances of the PAC motif.

   3.1. First, run your code on the provided sequence file PACPlusSeqs.fasta in the directory Testing. Use motif width 8, accuracy 0.01, and maximum iterations 100, True for includeReverseStrand, and 3 for numMotifsToFind. This may take about 5 minutes to run. If it runs for more than 30 minutes either you have an ancient computer or there is something wrong with your implementation. Examine your 3 outputs, first looking at the consensus (most probable base in each position) and then using prettyPrintPFM to look at the actual numbers.

   3.2. Now go to the MEME web site, upload the sequence file, change expected frequency to "one occurrence per sequence", number of motifs to 3, and under "Advanced Options" select "Search given strand only" and min and max motif widths both 8. Submit your job and wait a couple of minutes for the results to come up.
   
   3.3. How do your top 3 motifs compare to the top 3 found by the online implementation? Please consider which of the motifs are similar to each other, how similar they are, and how similar motifs were ranked among the 3 output by MEME and the 3 output by your code.

In [ ]:
from importlib.resources import path
from importlib import reload
import siteEM.assignment as assign
reload(assign) # This ensures code is reloaded each time
from siteEM.assignment import siteEM_intializer
with path('siteEM.data', 'PACPlusSeqs.fasta') as small_fasta_path:
            fasta_path = small_fasta_path

# Call siteEM-initializer and print out consensus strings for all 3 motifs.
# Then write some text to answer the question.

## sitePosterior

This assignment makes use of `sitePosterior()`, which you wrote for the previous
assignment. For convenience, a correct, obfuscated version is provided at the end
of the assignment.py file.

## Optional: Running from the command line.

You don't need to do this for this assignment, but if you are interested, once 
you have completed the assignments and all of the tests are passing, you can 
install the package in editable mode with:


```python
poetry run python -m pip install -e .
```

If this is successful, you should be able to run the script with:

```python
poetry run python -m siteEM --help
```

which will display a helpful dialogue on what parameters you may pass to the
script to execute siteEM on a FASTA file of your choice. For example, a
command to run siteEM on the PACPlusSeqs.fasta file with a motif_length of 8,
an accuracy threshold of 0.01, 100 iterations, 3 motifs, to write to a file
in the current working directory named pac_test.txt, and to include the
reverse complement of the sequences in the analysis would be:

```python
poetry run python -m siteEM -f siteEM/data/PACPlusSeqs.fasta -m 8 -a 0.01 -i 100 -n 3 -o pac_test.txt -r
```
